In [1]:
import numpy as np
import pprint
import sys
import warnings

sys.path.append('../src')
from simulation.user_sessions import simulate_users
from experiments.pre_test_analysis import required_sample_size, cuped_corr_coef
from experiments.experiment import experiment

In [2]:
cuped_corrcoef_list = []
for _ in range(5):
    cuped_corrcoef = cuped_corr_coef(100000, 8)
    cuped_corrcoef_list.append(cuped_corrcoef)

cuped_corrcoef_avg = np.mean(cuped_corrcoef_list)
print('Estimated CUPED correlation coefficient: {:.3f}'.format(cuped_corrcoef_avg))

pre_c_users, post_c_users, pre_t_users, post_t_users = simulate_users(100000,8,.03)
pre_exp_users_sum = np.vstack([pre_c_users, pre_t_users]).sum(axis=1)
post_exp_users_sum = np.vstack([post_c_users, post_t_users]).sum(axis=1)

t_test_n, t_test_cuped_n, seq_test_n, seq_test_cuped_n = required_sample_size(pre_exp_users_sum, cuped_corr_coef=cuped_corrcoef_avg, seq_final_alpha=.045)
print(f'T-test required sample size: {t_test_n}')
print(f'T-test with CUPED required sample size: {t_test_cuped_n}')
print(f'Group sequential test required sample size: {seq_test_n}')
print(f'Group sequential test with CUPED required sample size: {seq_test_cuped_n}')

Estimated CUPED correlation coefficient: 0.400
T-test required sample size: 19563
T-test with CUPED required sample size: 16437
Group sequential test required sample size: 20192
Group sequential test with CUPED required sample size: 16965


~.4 correlation coefficient between pre- and post-experiment sessions over a 4 week period seems reasonable. It creates a ~16% decrease in required sample size when using CUPED. Group sequential test is only slightly higher than t-test since I will be using the O'Brien-Flemming method.

In [8]:
exp = experiment(
    pre_c_user_sessions=pre_c_users,
    post_c_user_sessions=post_c_users,
    pre_t_user_sessions=pre_t_users,
    post_t_user_sessions=post_t_users,
    t_test_n=t_test_n,
    t_test_cuped_n=t_test_cuped_n,
    seq_test_n=seq_test_n,
    seq_test_cuped_n=seq_test_cuped_n,
)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    exp.create_test_groups()
    exp.create_test_entries()
    print(exp._ttest_cuped_t_sessions.sort_values('user_id'))
    print(exp._ttest_cuped_t_cov.sort_values('user_id'))
    print(exp._seq_cuped_t_sessions.sort_values('user_id'))
    print(exp._seq_cuped_t_cov.sort_values('user_id'))
    #exp.run_experiment()
    #pprint.pprint(exp.results)
    #exp._seq_cuped_c_sessions.describe()


       0  1  2   3  user_id  period_entry
2      2  1  1   2        2             4
4      2  4  1   3        4             4
5      1  1  0   3        5             4
13     0  0  6   1       13             3
14     3  1  2   1       14             3
...   .. .. ..  ..      ...           ...
49983  0  2  1   0    49983             2
49992  5  4  3  10    49992             2
49993  2  9  3   5    49993             3
49994  5  5  4   4    49994             1
49995  1  2  1   0    49995             2

[16437 rows x 6 columns]
       0  1  2   3  user_id  period_entry
0      1  1  1   7        2             4
1      6  7  3   7        4             4
2      0  2  1   1        5             4
3      0  1  0   0       13             3
4      1  1  1   1       14             3
...   .. .. ..  ..      ...           ...
16432  6  6  4  10    49983             2
16433  1  2  5   2    49992             2
16434  5  1  0   0    49993             3
16435  0  2  0   6    49994             1
16436  4